In [1]:
GENIUS_API_TOKEN='genius_credentials'

In [61]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

In [56]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
morlockk = request_song_url('ABBA', 2)
morlockk

Found 2 songs by ABBA


['https://genius.com/Abba-mamma-mia-lyrics',
 'https://genius.com/Abba-dancing-queen-lyrics']

In [101]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

In [104]:
# Get artist object from Genius API
def request_song_info(song_title, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': song_title}
    response = requests.get(search_url, data=data, headers=headers)
    return response

# Get song url form Genius API
def scrape_lyrics_genre(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = lyrics.replace('\n', ' ')
    return lyrics


# Get Genius.com lyrics from song object
def request_song_lyrics(song_title, artist_name):
    page = 1
    
    response = request_song_info(song_title, page)
    json = response.json()

    # Collect song objects
    song_info = {}
    song = json['response']['hits'][0]['result']
    if song_title.lower() in song['title'].lower() and artist_name.lower() in song['primary_artist']['name'].lower():
        url = song['url']
        lyrics = scrape_lyrics_genre(url)
        song_info[song_title] = lyrics
    print(url)    
    return song_info
    
# DEMO
napalm = request_song_lyrics('Napalmregen', 'Morlockk Dilemma')

https://genius.com/Morlockk-dilemma-napalmregen-lyrics


In [105]:
napalm

{'Napalmregen': "   Yo, ich bin zurück in dem Mutterfick Und Rapper wagen keinen Schritt ohne Schulterblick In meiner Gegend ist Leben nichts wert Und die Wege des Herrn führ'n in den Gegenverkehr  Ich treffe deine Freundin im Sommerkleid Näh' ihre Titten auf ihr Kreuz für den Doggy-Style Deine Sarah wird so brutal in den Mund gebumst Doch ich tat's für den Untergrund Sie gibt mir Schädel wie Guillotinen Auf dem Bettvorleger mit Tiermotiven Sieh', das sind niedere Triebe Und über ihrer Vagina klebt jetzt ein Kriposiegel Rapper werden geopfert für Luzifer Bin so abgehoben, ich boxe den Jupiter Zieh' eine Mac-10 im Lapdance-Club Der Letzte, der nach meinem Mic griff, war Captain Hook   Über der Szene kreisen Rabenvögel Stapelt die Leichen in den Straßengräben Ein kurzes Brennen in den Atemwegen Denn das ist Napalmregen über Ferropolis  Liebesgrüße aus der Stratosphäre Stapelt die Leichen in den Straßengräben Ein kurzes Brennen in den Atemwegen Denn das ist Napalmregen über Ferropolis   I

In [77]:
import lyricsgenius as lg

genius = lg.Genius('genius_credentials',  # Client access token from Genius Client API page
                             skip_non_songs=True,
                             remove_section_headers=True)

In [79]:
file = open("./lyrics_1.txt", "w")  # File to write lyrics to

artists = ['ABBA', 'Morlockk Dilemma', 'Apache 207']


def get_lyrics(arr, k):  # Write lyrics of k songs by each artist in arr
    c = 0  # Counter
    for name in arr:
        try:
            songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
            s = [song.lyrics for song in songs]
            file.write("\n \n   <|endoftext|>   \n \n".join(s))  # Deliminator
            c += 1
            print(f"Songs grabbed:{len(s)}")
        except:  #  Broad catch which will give us the name of artist and song that threw the exception
            print(f"some exception at {name}: {c}")


get_lyrics(artists, 3)

Searching for songs by ABBA...

Song 1: "Mamma Mia"
Song 2: "Dancing Queen"
Song 3: "The Winner Takes It All"

Reached user-specified song limit (3).
Done. Found 3 songs.
Songs grabbed:3
Searching for songs by Morlockk Dilemma...

Song 1: "Napalmregen"
Song 2: "Der Baum"
Song 3: "Portwein"

Reached user-specified song limit (3).
Done. Found 3 songs.
Songs grabbed:3
Searching for songs by Apache 207...

Song 1: "Roller"
Song 2: "Wieso tust Du dir das an?"
Song 3: "Fame"

Reached user-specified song limit (3).
Done. Found 3 songs.
Songs grabbed:3


In [119]:
import pandas as pd

In [130]:
lyrics = pd.DataFrame(columns=['title', 'lyrics'])

songs = ['Roller', 'Napalmregen', 'Mamma Mia']

def get_lyrics(df, arr):  # Write lyrics of k songs by each artist in arr
    for name in arr:
        l = genius.search_song(name).lyrics
        l = re.sub(r'[\(\[].*?[\)\]]', '', l)
        l = re.sub('\n', ' ', l)
        df = df.append({'title': name, 'lyrics': l}, ignore_index=True)
        
    return df

get_lyrics(lyrics, songs)

Searching for "Roller"...
Done.
Searching for "Napalmregen"...
Done.
Searching for "Mamma Mia"...
Done.


,title,lyrics
0,Roller,"Lucry Suena Warum fickt ihr Kopf, Kopf? Mein ..."
1,Napalmregen,"Yo, ich bin zurück in dem Mutterfick Und Rappe..."
2,Mamma Mia,I've been cheated by you since I don't know wh...


In [128]:
lyrics

,title,lyrics
